## Chroma Quick Start
docs: https://docs.trychroma.com/getting-started


In [ ]:
%pip install chromadb

In [2]:
import chromadb
chroma_client = chromadb.Client()

In [3]:
# create collection
# Collections are where you'll store your embeddings, documents, and any additional metadata. 
# You can create a collection with a name:

collection = chroma_client.create_collection(name='my_collection')

In [4]:
# add some text
# Chroma将存储你的文本，并自动处理标记化、嵌入和索引。

collection.add(
    documents=["This is a document", "This is another document"],
    metadatas=[{"source": "my_source"}, {"source": "my_source"}],
    ids=["id1", "id2"]
)

In [7]:
# 相似度查询
collection.query(query_texts=["This is a document"], n_results=1)

{'ids': [['id1']],
 'embeddings': None,
 'documents': [['This is a document']],
 'metadatas': [[{'source': 'my_source'}]],
 'distances': [[0.0]]}

In [11]:
# collection.add(
#     documents=["王存是个好人", "方朋可真行啊"],
#     metadatas=[{"source": "other"}, {"source": "other"}],
#     ids=["id3", "id4"]
# )
collection.query(query_texts=["谁是好人?"], n_results=1)

{'ids': [['id3']],
 'embeddings': None,
 'documents': [['王存是个好人']],
 'metadatas': [[{'source': 'other'}]],
 'distances': [[0.7229322195053101]]}

In [13]:
chroma_client.get_or_create_collection(name='my_collection_v2')
chroma_client.list_collections()

[Collection(name=my_collection), Collection(name=my_collection_v2)]

In [14]:
collection.count()

4

In [15]:
collection.add(
    ids=["id101", "id102", "id103", "id104"],
    metadatas=[{"name": "101"}, {"name": "102"}, {"name": "103"}, {"name": "104"}],
    documents=["101进入教室开始上课", "出现了网络卡顿的问题，在102学生上", "打开摄像头进行调试103即可", "教室里看不到学生 104"]
)

In [16]:
collection.count()

8

In [20]:
collection.get(ids=["id103", "id102"], limit=3)

{'ids': ['id103', 'id102'],
 'embeddings': None,
 'documents': ['打开摄像头进行调试103即可', '出现了网络卡顿的问题，在102学生上'],
 'metadatas': [{'name': '103'}, {'name': '102'}]}

In [22]:
collection.get(where={"name": "102"}) # where filter metabases

{'ids': ['id102'],
 'embeddings': None,
 'documents': ['出现了网络卡顿的问题，在102学生上'],
 'metadatas': [{'name': '102'}]}

In [23]:
collection.get(where_document={"$contains": "教室"})

{'ids': ['id101', 'id104'],
 'embeddings': None,
 'documents': ['101进入教室开始上课', '教室里看不到学生 104'],
 'metadatas': [{'name': '101'}, {'name': '104'}]}

In [ ]:
collection.peek(limit=3)
# output
# {'ids': ['id1', 'id2', 'id3'],
#  'embeddings': [[-0.06602787971496582,
#    0.17150495946407318,
#    ...

In [35]:
collection.query(query_texts=["教室"], n_results=2, where={'name': '101'}, include=['documents','metadatas','distances'])

{'ids': [['id101']],
 'embeddings': None,
 'documents': [['101进入教室开始上课']],
 'metadatas': [[{'name': '101'}]],
 'distances': [[1.0058444738388062]]}

In [36]:
collection.query(query_texts=["教室"], n_results=2)

{'ids': [['id103', 'id101']],
 'embeddings': None,
 'documents': [['打开摄像头进行调试103即可', '101进入教室开始上课']],
 'metadatas': [[{'name': '103'}, {'name': '101'}]],
 'distances': [[0.8989331722259521, 1.0058444738388062]]}

## 关于向量查询解释

对于上面的查询，我问GPT4给我一个解释：

> 我的问题是:我在进行向量查询 “教室” 关键字时(`collection.query(query_texts=["教室"], n_results=2)`), 
> 其中distances表示什么？ 为什么 我期望的 “教室里看不到学生 104” 没有匹配上？而“打开摄像头进行调试103即可” 匹配上了。

GPT给的回答：

> distances 字段代表查询向量与结果向量之间的距离。这个距离通常使用某种形式的距离度量（例如欧氏距离或余弦相似度）来计算。
> 距离越小，表示查询向量与结果向量越相似，因此在大部分场景下，我们认为距离越小，查询的结果越相关。

> 然后，你提到你期望的 "教室里看不到学生 104" 没有被匹配上，而 "打开摄像头进行调试103即可" 被匹配上了。
> 这可能是因为你用来表示文档的向量模型（比如BERT，Word2Vec等）在生成 "教室" 这个词的向量表示时，与 "打开摄像头进行调试103即可" 这个文档的向量更接近。

> 这可能是因为模型在学习过程中，找到了"教室"和"打开摄像头进行调试"之间的某种关联，比如它们都可能在某些上下文中关联到类似的主题或情境。而"教室里看不到学生"这个文档可能与模型学习到的"教室"的上下文关联性较低。

> 需要注意的是，向量搜索是基于语义的，而不仅仅是基于关键词的。即使文档中并未直接出现查询的关键词，只要其语义上与查询的关键词接近，也可能被搜索到。

> 此外，向量空间模型的效果很大程度上取决于训练数据和训练方法。如果你发现模型的表现不如预期，你可以尝试使用不同的模型，或者使用更多或更贴近你的实际需求的训练数据来训练模型。

![](https://limbo.oss-cn-beijing.aliyuncs.com/wiki/202306281832436.png)

In [37]:
collection.query(query_texts=["上课过程中的各种问题"], n_results=2)

{'ids': [['id102', 'id101']],
 'embeddings': None,
 'documents': [['出现了网络卡顿的问题，在102学生上', '101进入教室开始上课']],
 'metadatas': [[{'name': '102'}, {'name': '101'}]],
 'distances': [[0.6690373420715332, 0.6732606887817383]]}

In [38]:
collection.update(ids=["id101"], documents=["101进入教室开始上课啦"], metadatas=[{"name": "101111"}])

In [39]:
collection.get(ids=['id101'])

{'ids': ['id101'],
 'embeddings': None,
 'documents': ['101进入教室开始上课啦'],
 'metadatas': [{'name': '101111'}]}

In [40]:
collection.upsert(ids=["id101"], documents=["101进入教室开始上课啦~"], metadatas=[{"name": "1011110", 'flag': 'test'}])
collection.upsert(ids=["id108"], documents=["108听不到老师说话"], metadatas=[{"name": "108", 'flag':'test'}])

collection.get(where={'flag': 'test'})

{'ids': ['id101', 'id108'],
 'embeddings': None,
 'documents': ['101进入教室开始上课啦~', '108听不到老师说话'],
 'metadatas': [{'name': '1011110', 'flag': 'test'},
  {'name': '108', 'flag': 'test'}]}

In [41]:
collection.delete(ids=["id108"])